In [1]:
day = 12
from IPython.display import IFrame
url = "https://adventofcode.com/2022/day/%d"%(day)
IFrame(src=url, width=1000, height=600)

In [2]:
import urllib.request
import sys
from timeit import default_timer as timer
import itertools


def gather_input_data(url, sessionId, transform=lambda x: str(x, "utf-8").strip('\n')):
    request = urllib.request.Request(url)
    request.add_header("cookie", "session={}".format(sessionId)) # Source the data directly from AoC

    values = []
    with urllib.request.urlopen(request) as data:
        for line in data:
            values.append(transform(line))

    return values

def get_data(day, year=2022):
    with open('sessionID') as f:
        sessionId = f.readlines()[0]
    url = "https://adventofcode.com/%d/day/%d/input"%(year,day)
    data = gather_input_data(url, sessionId)
    return data


def parse_line(line):
    return [ord(letter) - ord('a') + 1 for letter in line]
    

def read_file(filename):
    # read file
    with open(filename) as f:
        lines = f.readlines()
    output = [str.rstrip(line) for line in lines]    
    return output



In [28]:

def get_node(val, all_nodes=all_nodes):
    return [list(all_nodes.values()).index(val)]

def neighbors(node):
    directions = [[1, 0], [0, 1], [-1, 0], [0, -1]]
    result = []
    for direction in directions:
        neighbor = (node[0] + direction[0], node[1] + direction[1])
        if neighbor in all_nodes.values():
            result.append(neighbor)
    return result

def state_transition(graph, row_offset):        
    for row in range(graph.shape[0]-row_offset):
        # a -> b
        a = all_nodes[row]
        b = all_nodes[row+row_offset]
        if(data[b] <= data[a] + 1):
            if b in neighbors(a):
                graph[row, row+row_offset] = 1
                #print('a',a,b,row,row+row_offset,data[a], data[b])
                
        # b -> a
        if(data[a] <= data[b] + 1):
            if a in neighbors(b):
                graph[row+row_offset, row] = 1
                #print('b',a,b,row,row+row_offset,data[a], data[b])

    return graph

In [31]:
data = get_data(day)
# data = [\
# 'Sabqponm',
# 'abcryxxl',
# 'accszExk',
# 'acctuvwj',
# 'abdefghi']

data = [parse_line(line) for line in data]
import numpy as np
data = np.array(data)


## grid to graph with a view to Djkstra
all_nodes = {}
count = 0
for x in range(data.shape[0]):
    for y in range(data.shape[1]):
        all_nodes[count] = (x, y)
        count += 1
graph = np.zeros([len(all_nodes), len(all_nodes)]).astype(int)

def get_node(val, all_nodes=all_nodes):
    return list(all_nodes.keys())[list(all_nodes.values()).index(val)]


start = np.where(data==-13)
end = np.where(data==-27)
start = (start[0][0], start[1][0])
end = (end[0][0], end[1][0])
data[start] = 1
data[end] = 26
start_node = get_node(start)
end_node = get_node(end)

In [55]:
offset_node = neighbors((0,0))
graph = state_transition(graph, get_node(offset_node[0]))
graph = state_transition(graph, get_node(offset_node[1]))

from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import dijkstra, shortest_path

def get_path(Pr, i, j):
    path = [j]
    k = j
    while Pr[i, k] != -9999:
        path.append(Pr[i, k])
        k = Pr[i, k]
    return path[::-1]

start_timer = timer()
D, Pr = shortest_path(csr_matrix(graph), directed=True, method='D', return_predecessors=True)

print("Completed in %fms" % ((timer() - start_timer) * 1000))

part1 = (len(get_path(Pr, start_node,end_node))-1)

(row, col) = np.where(data==1)
paths = [len(get_path(Pr, get_node(a),end_node))-1 for a in zip(row,col)]
part2 = min([path for path in paths if path>0])
print(part1, part2)

Completed in 4474.094436ms
456 454
